In [28]:
from __future__ import division, print_function
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# A sketch of the task


# rows are features, columns are sequence elements

In [30]:
# where are there parameters?

In [31]:
# With theano.tensor.nnet.conv2d
import theano.tensor as T
from theano.tensor import nnet
from theano import function

In [32]:
# 3 features, filter size =2
# A = [[1,1,1,1], [2,2,2,2], [3,3,3,3]]
# B = [[1,1],[2,2], [3,3]]

# filter dim should be (3,3, 1, 2)
b_filter = np.array([ [ [[1,1]], [[0,0]], [[0,0]] ], [ [[0,0]],[[2,2]], [[0,0]] ], [ [[0,0]],[[0,0]], [[3,3]] ] ], dtype='float32')

sequence_1d_data=T.tensor4(name='sequence_1D')
# filters = tensor4(nb filters, stack size, nb row, nb col)
print(b_filter.shape)

conv_out = nnet.conv.conv2d(sequence_1d_data.dimshuffle(1, 0, 2, 3), b_filter, border_mode='full')

(3, 3, 1, 2)


In [33]:
convolve_1d = function([sequence_1d_data], conv_out)

In [34]:
# so that h_shuffled (A_shuffled?) is (1, ndim, 1, nitems)
# x1 = np.array([ [[ [1,2,3],[1,2,3],[1,2,3],[1,2,3] ]], [[ [1,2,3],[1,2,3],[1,2,3],[1,2,3] ]], [[ [1,2,3],[1,2,3],[1,2,3],[1,2,3] ]] ], dtype='float32')
x1 = np.array([ [[ [1,2,3,4] ]], [[ [1,2,3,4] ]], [[ [1,2,3,4] ]] ], dtype='float32') 
x1_conv = convolve_1d(x1)

x1_conv

array([[[[  1.,   3.,   5.,   7.,   4.]],

        [[  2.,   6.,  10.,  14.,   8.]],

        [[  3.,   9.,  15.,  21.,  12.]]]], dtype=float32)

In [35]:
# trying to do a k max

from theano.tensor import argsort

# TODO - flatten out the unneccesary dims here
sort_test = argsort(conv_out, axis=3)

In [36]:
# slice the end of sort test to get the top K column indices
k = 3
top_indices = sort_test[:,:,:,-k:].reshape((3,3))

# we want to sort the _INDICES_ now, so that the original order is maintained


In [37]:
from theano.tensor import sort
ordered_top_indices = sort(top_indices)

In [38]:
# flatten out those extra dims for sanity -- note we cannot hard code the shape in reality
just_data = conv_out.reshape((3,5))

In [39]:
# now take those values from the original, and make a new tensor with only the top K
from theano.tensor import take

# weird hack because of the way the 'take' function works
linear_k_max = take(just_data, ordered_top_indices, axis=1)[:,0,:]

In [40]:
from theano.tensor.nnet import sigmoid

# TODO: add biases for each row
feature_map_1 = sigmoid(linear_k_max)

In [41]:
from theano.tensor import sum

folded_k_max = sum(linear_k_max, axis=1)

In [ ]:
# a convolutional layer

In [ ]:
# a k-max layer

In [42]:
# max_test = function([sequence_1d_data], top_indices)
# max_test = function([sequence_1d_data], just_data)
max_test = function([sequence_1d_data], linear_k_max)
max_test = function([sequence_1d_data], conv_out)
max_test = function([sequence_1d_data], folded_k_max)
max_test = function([sequence_1d_data], feature_map_1)

/home/chris/programs/anaconda/lib/python2.7/site-packages/theano/tensor/subtensor.py:110: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  start in [None, 0] or
/home/chris/programs/anaconda/lib/python2.7/site-packages/theano/tensor/subtensor.py:114: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  stop in [None, length, maxsize] or


In [43]:
max_test(x1)

array([[ 0.99330717,  0.99908894,  0.98201376],
       [ 0.99995458,  0.99999917,  0.99966466],
       [ 0.9999997 ,  1.        ,  0.99999386]], dtype=float32)

In [27]:
# the convolution operation

In [2]:
# the 1D convolution

In [3]:
# the intuition behind the convolutional sentence model

In [ ]:
# K-max pooling

questions & exercises

(1) write a function to get the _indices_ of the top K values in an array
(2) write a 1d narrow and wide convolution using python (and numpy if you want)
